In [28]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import lmdb

# Add paths
sys.path.insert(0, os.path.abspath('../../../python/')) # PetaVision
sys.path.insert(0, os.path.abspath(os.environ['HOME']+'/Work/Libraries/caffe/python/')) # Caffe

# Import aux libraries
from pvtools import *
import caffe

In [32]:
#set up path to pvp file
pvp_file_path = os.environ['HOME']+'/Work/LANL/Data/S1.pvp'

pvData = readpvpfile(pvp_file_path, 1, 10, 0, 1)

File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 1 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 2 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 3 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 4 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 5 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 6 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 7 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 8 of 10
File /Users/dpaiton/Work/LANL/Data/S1.pvp: frame 9 of 10


In [74]:
#So first column are indices, second activities. Indices are rolled out, ticks f,x,y, fastest to slowest
#dimensions are (32/4) x (32/4) x 192
#that's 192 features, cifar 32x32 dimensions divided by stride of 4
#the matrix is organized (num_non_zero_elements, 2) where the second index gives [dense_idx, activity]


#below code fails because python only supports 2-D sparse matrices.
#So I can run this code per frame and recompile it into a 3-D dense matrix, or I can find a better solution...
from scipy.sparse import coo_matrix

numIm = len(pvData)
numF = pvData.header['nf']
numY = pvData.header['ny']
numX = pvData.header['nx']

out_mat = np.zeros((numIm, numF, numY, numX))

for frame_idx in range(numIm):
    frame_data = np.array(pvData[frame_idx].values)

    i = np.zeros(frame_data.shape[0])
    j = np.zeros(frame_data.shape[0])
    for idx in range(px_shape.shape[0]):
        i[idx] = frame_data[idx,0].astype('int') / numY # row
        j[idx] = frame_data[idx,0].astype('int') % numY # col

    out_mat[frame_idx,feat_idx,:,:] = np.array(coo_matrix((sub_mat[:,1], (i,j))).todense()).shape


In [79]:
pvData.header


{'datasize': 8,
 'datatype': 4,
 'filetype': 6,
 'headersize': 80,
 'kx0': 0,
 'ky0': 0,
 'nbands': 150000,
 'nbatch': 1,
 'nf': 192,
 'numparams': 20,
 'numrecords': 1,
 'nx': 8,
 'nxGlobal': 8,
 'nxprocs': 1,
 'ny': 8,
 'nyGlobal': 8,
 'nyprocs': 1,
 'recordsize': 2371,
 'time': 400.0}